In [28]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [2]:
answers = {}

In [3]:
# Load dataframe
data_january = 'fhv_tripdata_2021-01.parquet'
df = pd.read_parquet('./data/' + data_january)

In [4]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [5]:
# Question 1: Read the data for January. How many records are there?
answers['q1'] = df.shape[0]
answers

{'q1': 1154112}

In [6]:
# calculate duration and add as columns
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [7]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [8]:
# Question 2: What's the average trip duration in January?

answers['q2'] =  df.duration.mean()
answers

{'q1': 1154112, 'q2': 19.167224093791006}

In [9]:
df['duration'].describe(percentiles=[.90, .95, .97]).apply(lambda x: format(x, 'f'))

count    1154112.000000
mean          19.167224
std          398.692165
min            0.016667
50%           13.400000
90%           35.633333
95%           47.250000
97%           57.750000
max       423371.050000
Name: duration, dtype: object

In [10]:
rows_before_dropping = df.shape[0]
df = df[ (df['duration'] >= 1) & (df['duration'] <= 60)]
rows_after_dropping = df.shape[0]

print(f'Dropped {rows_before_dropping - rows_after_dropping} rows')
df.head()

Dropped 44286 rows


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000


In [11]:
# Question 3: What's the factions of missing values for the pickup location ID? (Or the fraction of "-1"s after you filled the NAs)

nonnull_values_count = df['PUlocationID'].count()
null_values_count = df['PUlocationID'].isnull().sum()

null_values_count / (nonnull_values_count + null_values_count)


0.8352732770722617

In [12]:
df['PUlocationID'].fillna(-1, inplace=True)
df['DOlocationID'].fillna(-1, inplace=True)

In [13]:
missing_values = (df['PUlocationID'] == -1).sum() /df['PUlocationID'].count()
answers['q3'] = missing_values
answers

{'q1': 1154112, 'q2': 19.167224093791006, 'q3': 0.8352732770722617}

In [36]:
train_dicts = df[['PUlocationID', 'DOlocationID']].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values
dim = X_train.get_shape()[1]
answers['q4'] = dim
answers

{'q1': 1154112, 'q2': 19.167224093791006, 'q3': 0.8352732770722617, 'q4': 2}

In [38]:
# train lineair regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)

answers['q5'] = rmse
answers

{'q1': 1154112,
 'q2': 19.167224093791006,
 'q3': 0.8352732770722617,
 'q4': 2,
 'q5': 11.415432830521663}

# Validation

In [40]:
data_february = 'fhv_tripdata_2021-02.parquet'
df_val = pd.read_parquet('./data/' + data_february)

In [41]:
def preprocess(df):
    # calculate duration and add as columns
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    # filter
    df = df[ (df['duration'] >= 1) & (df['duration'] <= 60)]

    # replace nans
    df['PUlocationID'].fillna(-1, inplace=True)
    df['DOlocationID'].fillna(-1, inplace=True)
    
    return df


In [42]:
df_val = preprocess(df_val)
df_val

/var/folders/y3/t8496fv95p9_rxm7q0b2_929rg90j4/T/ipykernel_40889/4189926640.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'].fillna(-1, inplace=True)
/var/folders/y3/t8496fv95p9_rxm7q0b2_929rg90j4/T/ipykernel_40889/4189926640.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'].fillna(-1, inplace=True)


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1.0,225.0,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1.0,61.0,None,B00037,8.966667
...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1.0,31.0,None,B01717,13.533333
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1.0,169.0,None,B01717,11.466667
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285,25.383333
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285,18.050000


In [44]:
val_dicts = df_val[['PUlocationID', 'DOlocationID']].to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)
y_val = df_val['duration'].values

y_val_pred = lr.predict(X_val)

rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)

answers['q6'] = rmse_val
answers


{'q1': 1154112,
 'q2': 19.167224093791006,
 'q3': 0.8352732770722617,
 'q4': 2,
 'q5': 11.415432830521663,
 'q6': 11.85822362355935}